In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import distill

In [3]:
from smart_open import open as smart_open
from transformers import BertTokenizer
import torch
import io
from dataloader import get_dataloader, check_cache, prepare_features, process_data, prepare_inputs
from load import load_data
import os

In [4]:
load_path = "https://amazonmassive.s3.us-west-1.amazonaws.com/model.pt"
print(load_path)
with smart_open(load_path, 'rb') as f:
    #buffer = io.BytesIO(f.read())
    model=torch.load(io.BytesIO(f.read()),map_location=torch.device('cuda'))

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", truncation_side="left")

# move elsewhere
config = {
        'vocab_size' : len(tokenizer.get_vocab()),
        'embedding_size' : 300,
        'hidden_size' : 512,
        'fc_size' : 128,
        'num_layers' : 2,
        'n_classes' : 60,
        'dropout' : 0.5,
        'epochs' : 40,
        'lr' : 5e-4,
        'temp' : 1,
        'weight_decay' : 1e-4,
        'alpha' : 0.95,
        'batch_size' : 256,
        'input_dir' : 'assets',
        'dataset' : 'amazon',
        'ignore_cache' : False,
        'max_len' : 20,
        'early_stop' : 3
        #'output_dir' : 'result',
        }

https://amazonmassive.s3.us-west-1.amazonaws.com/model.pt


In [ ]:
config

In [5]:
class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
config = AttributeDict(config)

In [6]:
cache_results, already_exist = check_cache(config)

if already_exist:
    print('exist')
    features = cache_results
else:
    print('not exist')
    data = load_data()
    features = prepare_features(config, data, tokenizer, cache_results)
datasets = process_data(config, features, tokenizer)

Creating new input features ...
exist


No config specified, defaulting to: amazon_massive_intent/en
Found cached dataset amazon_massive_intent (C:/Users/Wang/.cache/huggingface/datasets/mteb___amazon_massive_intent/en/1.0.0/a9ab9f5d309356e4995ca161846b3636f56df1ad43cb04bb300dd8b4f99141f4)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'label_text', 'text'],
        num_rows: 11514
    })
    validation: Dataset({
        features: ['id', 'label', 'label_text', 'text'],
        num_rows: 2033
    })
    test: Dataset({
        features: ['id', 'label', 'label_text', 'text'],
        num_rows: 2974
    })
})


100%|██████████████████████████████████████████████████████████████████████████| 11514/11514 [00:03<00:00, 3077.37it/s]


{'input_ids': [101, 4638, 10373, 2013, 2198, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]} {'id': '17180', 'label': 44, 'label_text': 'email_query', 'text': 'check email from john'}
Number of train features: 11514


100%|████████████████████████████████████████████████████████████████████████████| 2033/2033 [00:00<00:00, 3121.04it/s]


{'input_ids': [101, 2054, 1005, 1055, 1996, 4769, 2005, 4074, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]} {'id': '17167', 'label': 17, 'label_text': 'email_querycontact', 'text': "what's the address for alex"}
Number of validation features: 2033


100%|████████████████████████████████████████████████████████████████████████████| 2974/2974 [00:00<00:00, 3162.11it/s]

{'input_ids': [101, 2038, 2198, 2741, 2033, 2019, 10373, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]} {'id': '17179', 'label': 44, 'label_text': 'email_query', 'text': 'has john sent me an email'}
Number of test features: 2974


In [7]:
distill.learn(config, model, datasets , tokenizer)

Loaded train data with 45 batches


100%|███████████████████████████████████████████████████████████████████████| 45/45 [00:15<00:00,  2.83it/s, loss=3.32]


epoch 0 | train losses 3.4368905120425755
Loaded validation data with 8 batches


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 14.00it/s]


validation acc: 0.06443679291687161 |dataset split validation size: 2033


...

100%|█████████████████████████████████████████████████████████████████████| 45/45 [00:20<00:00,  2.25it/s, loss=0.0791]


epoch 39 | train losses 0.11444978747102949
Loaded validation data with 8 batches


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 10.34it/s]

validation acc: 0.794392523364486 |dataset split validation size: 2033


'end of epochs'

In [10]:
student_model = distill.StudentModel(config)
student_model.load_state_dict(torch.load('./saved_model/student.pt'))
student_model.eval()


StudentModel(
  (embedding): Embedding(30522, 300)
  (rnn): LSTM(300, 512, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=128, out_features=60, bias=True)
  )
)

In [11]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

In [12]:
optimizer = optim.AdamW(student_model.parameters(), lr=config.lr, weight_decay = config.weight_decay)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0, T_mult)
for i in range(20):
    scheduler.step()
    scheduler.step(26)
    scheduler.step()

NameError: name 'optimizer' is not defined

In [ ]:
a = torch.randint(0, 5, size = (256 , 20 , 1024))
a

In [ ]:
a[:, -1, :]

In [ ]:
length = torch.randint(1, 20, size = (256,))
length

In [ ]:
sum(length)

In [ ]:
a[:, -1, :].size()

In [ ]:
packed = torch.nn.utils.rnn.pack_padded_sequence(a, length, batch_first=True, enforce_sorted = False)

In [ ]:
packed.data.size()